Ang, Brayden Jansen O.

Zialcita, Andrea Mikaela S.

---

CSCI 111 Final Project

8-Puzzle Problem

In [ ]:
import numpy as np

In the 8-puzzle problem, we represent a state as a complete 3 by 3 NumPy array containing each of `0, 1, 2, ..., 8` exactly once. The numbers `1, 2, ..., 8` represent tile 1, tile 2, ..., tile 8, respectively, while the number `0` represents the empty space, which we will refer to as the zero tile.

The goal state, or the state we want to achieve, is then given as:

In [ ]:
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8]
goal_state = np.array(numbers).reshape(3, 3)
goal_state

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

We then define operations on a given state that represent possible actions to transform the state into another state. In any setup of the board, we can move at most four different tiles, namely the ones to the left of, above, to the right of, and below the zero tile, to the zero tile, effectively swapping their positions.

We use `swap_left` to denote the action of swapping the zero tile with the tile to its left (if it exists), `swap_up` to denote the action of swapping the zero tile with the tile above it (if it exists), and so on.

In [ ]:
def swap_left(state):
  if 0 not in state[:, 0]:
    new_state = state.copy()
    r, c = np.where(new_state == 0)
    new_state[r[0], c[0]], new_state[r[0], c[0] - 1] = new_state[r[0], c[0] - 1], new_state[r[0], c[0]]
    return new_state
  else:
    return None

def swap_up(state):
  if 0 not in state[0]:
    new_state = state.copy()
    r, c = np.where(new_state == 0)
    new_state[r[0], c[0]], new_state[r[0] - 1, c[0]] = new_state[r[0] - 1, c[0]], new_state[r[0], c[0]]
    return new_state
  else:
    return None

def swap_right(state):
  if 0 not in state[:, 2]:
    new_state = state.copy()
    r, c = np.where(new_state == 0)
    new_state[r[0], c[0]], new_state[r[0], c[0] + 1] = new_state[r[0], c[0] + 1], new_state[r[0], c[0]]
    return new_state
  else:
    return None

def swap_down(state):
  if 0 not in state[2]:
    new_state = state.copy()
    r, c = np.where(new_state == 0)
    new_state[r[0], c[0]], new_state[r[0] + 1, c[0]] = new_state[r[0] + 1, c[0]], new_state[r[0], c[0]]
    return new_state
  else:
    return None

We also define other utility functions that will be used in the search algorithms.

In [ ]:
def equal(array1, array2):
  return np.array_equal(array1, array2)
# Check if two NumPy arrays are equal (same shape and entries).

def array_in_dict(array, dictionary):
    flattened_array = tuple(array.flatten())
    if flattened_array in dictionary:
        return True
    return False
# Check if an array is a key in a given dictionary.

def is_solvable(state):
    flat_state = state.flatten()
    inversions = 0
    for i in range(len(flat_state)):
        for j in range(i + 1, len(flat_state)):
            if flat_state[i] > 0 and flat_state[j] > 0 and flat_state[i] > flat_state[j]:
                inversions += 1
    return inversions % 2 == 0
# Check if a given state is solvable.

def get_neighbors(state):
    neighbors = []
    for move_function in [swap_left, swap_up, swap_right, swap_down]:
        neighbor = move_function(state)
        if neighbor is not None:
            neighbors.append(neighbor)
    return neighbors
# Generate all possible neighbors for the current state using swap functions.

def find_path(initial_state: np.array, successor_list: list):
  path = [goal_state]
  state = goal_state
  while not equal(state, initial_state):
    for pair in successor_list:   # The successor_list will contain (parent, child) pairs. We use this to jump backwards (step-by-step) from the goal state to the initial state.
      if (pair[1] == state).all():
        path.append(pair[0])
        state = pair[0]
        break
  path.reverse()
  return tuple(path)
# Reconstruct the path from the initial state to the goal state.

# Breadth-First Search (BFS)

In breadth-first search, we traverse the search tree by first inspecting nodes that are directly successors/neighbors of the currently inspected node. So, our frontier is a queue, or a "first in, first out" frontier.

Note that the BFS algorithm below, if the given initial state is solvable, returns a triple containing the path from the initial state to the goal state, the length of that path (how many moves must be made to reach the goal state), and the number of nodes the algorithm had to inspect before finding the solution.

In [ ]:
from collections import deque

def bfs(start):
    if not is_solvable(start):
        return "This puzzle is not solvable."

    queue = deque([start])
    visited = {}
    successor_list = []
    nodes_inspected = 0

    while queue:
        current_state = queue.popleft()
        nodes_inspected += 1

        # mark the current state as visited
        visited[tuple(current_state.flatten())] = True

        # check if the current state is the goal state
        if equal(current_state, goal_state):
            print("Solvable!")
            path = find_path(start, successor_list)
            return (path, len(path), nodes_inspected)

        for neighbor in get_neighbors(current_state):
            if not array_in_dict(neighbor, visited):  # check against visited dictionary
                queue.append(neighbor)
                successor_list.append((current_state, neighbor))

    return "No solution found."

## Test (Solvable)

In [ ]:
solvable_state = np.array([[1, 8, 2],
                        [0, 4, 3],
                        [7, 6, 5]])

bfs(solvable_state)

Solvable!


((array([[1, 8, 2],
         [0, 4, 3],
         [7, 6, 5]]),
  array([[1, 8, 2],
         [4, 0, 3],
         [7, 6, 5]]),
  array([[1, 0, 2],
         [4, 8, 3],
         [7, 6, 5]]),
  array([[0, 1, 2],
         [4, 8, 3],
         [7, 6, 5]]),
  array([[4, 1, 2],
         [0, 8, 3],
         [7, 6, 5]]),
  array([[4, 1, 2],
         [7, 8, 3],
         [0, 6, 5]]),
  array([[4, 1, 2],
         [7, 8, 3],
         [6, 0, 5]]),
  array([[4, 1, 2],
         [7, 0, 3],
         [6, 8, 5]]),
  array([[4, 1, 2],
         [7, 3, 0],
         [6, 8, 5]]),
  array([[4, 1, 0],
         [7, 3, 2],
         [6, 8, 5]]),
  array([[4, 0, 1],
         [7, 3, 2],
         [6, 8, 5]]),
  array([[4, 3, 1],
         [7, 0, 2],
         [6, 8, 5]]),
  array([[4, 3, 1],
         [0, 7, 2],
         [6, 8, 5]]),
  array([[0, 3, 1],
         [4, 7, 2],
         [6, 8, 5]]),
  array([[3, 0, 1],
         [4, 7, 2],
         [6, 8, 5]]),
  array([[3, 1, 0],
         [4, 7, 2],
         [6, 8, 5]]),
  array(

## Test (Not Solvable)

In [ ]:
unsolvable_state = np.array([[8, 1, 2],
                        [0, 4, 3],
                        [7, 6, 5]])

bfs(unsolvable_state)

'This puzzle is not solvable.'

# Greedy Best-First Search

Another search algorithm we can use is the greedy best-first search. We define the heuristic function to be the total Manhattan distance of a given state, that is, the sum of how many moves each tile is from its correct position.

Our frontier is then a priority queue. After every node expansion, we sort the frontier by increasing total Manhattan distance so that the algorithm prioritizes inspecting nodes with less total Manhattan distance. This is because we are interpreting states with less total Manhattan distance to be "closer" to the goal state, since intuitively, it will take less moves to correct the position of each tile.

If the initial state is solvable, the algorithm below will again return a triple containing the path, the length of the path, and how many nodes were inspected before finding the path.

In [ ]:
def correct_position(n):
  if n == 1:
    return (0, 1)
  elif n == 2:
    return (0, 2)
  elif n == 3:
    return (1, 0)
  elif n == 4:
    return (1, 1)
  elif n == 5:
    return (1, 2)
  elif n == 6:
    return (2, 0)
  elif n == 7:
    return (2, 1)
  elif n == 8:
    return (2, 2)
  else:
    return
# Define the correct position of each tile.

def manhattan_distance(state: np.array):
  m = 0
  for row in state:
    for number in row:
      if number != 0:   # We do not include the Manhattan distance of the zero tile, since it is not technically a tile.
        r, c = np.where(state == number)
        r0, c0 = correct_position(number)[0], correct_position(number)[1]
        m += abs(r[0] - r0) + abs(c[0] - c0)
  return m
# Take the absolute value of the difference between a tile and its correct position's row and column number and sum over all tiles in a given state.

def sort_frontier(frontier: dict):
  sorted_items = sorted(frontier.items(), key=lambda item: item[1])
  frontier.clear()
  frontier.update(sorted_items)
# Sort the frontier by increasing total Manhattan distance.

def greedy(initial_state):
  if not is_solvable(initial_state):
    return "The goal state is not reachable."
  # Frontier is a dictionary so that we can sort states by the heuristic function.
  frontier = {tuple(initial_state.flatten()) : manhattan_distance(initial_state)}
  visited = {}
  successor_list = []

  # Count how many nodes are inspected.
  nodes_inspected = 0

  if equal(initial_state, goal_state):
    return tuple([initial_state])
  node = initial_state

  while not equal(node, goal_state):
    # Record the node as visited.
    visited[tuple(node.flatten())] = True

    nodes_inspected += 1

    # Remove the currently inspected node from the frontier.
    frontier.pop(tuple(node.flatten()))

    # Expand the currently inspected node.
    successors = get_neighbors(node)
    for successor in successors:
      if not array_in_dict(successor, visited) and not array_in_dict(successor, frontier):
        successor_list.append((node, successor))
        frontier[tuple(successor.flatten())] = manhattan_distance(successor)

    # Sort the frontier by total Manhattan distance.
    sort_frontier(frontier)

    # print(frontier)

    # Inspect the next node in the frontier after sorting.
    node = np.array(next(iter(frontier))).reshape(3, 3)

  # Trace back the path found from the initial state to the goal state.
  path = find_path(initial_state, successor_list)
  return (path, len(path), nodes_inspected)
# You may wish to print the frontier after each node expansion to examine how the algorithm sorts the frontier by total Manhattan distance.

In [ ]:
solvable_state

array([[1, 8, 2],
       [0, 4, 3],
       [7, 6, 5]])

In [ ]:
greedy(solvable_state)

((array([[1, 8, 2],
         [0, 4, 3],
         [7, 6, 5]]),
  array([[1, 8, 2],
         [4, 0, 3],
         [7, 6, 5]]),
  array([[1, 8, 2],
         [4, 3, 0],
         [7, 6, 5]]),
  array([[1, 8, 2],
         [4, 3, 5],
         [7, 6, 0]]),
  array([[1, 8, 2],
         [4, 3, 5],
         [7, 0, 6]]),
  array([[1, 8, 2],
         [4, 3, 5],
         [0, 7, 6]]),
  array([[1, 8, 2],
         [0, 3, 5],
         [4, 7, 6]]),
  array([[1, 8, 2],
         [3, 0, 5],
         [4, 7, 6]]),
  array([[1, 0, 2],
         [3, 8, 5],
         [4, 7, 6]]),
  array([[0, 1, 2],
         [3, 8, 5],
         [4, 7, 6]]),
  array([[3, 1, 2],
         [0, 8, 5],
         [4, 7, 6]]),
  array([[3, 1, 2],
         [4, 8, 5],
         [0, 7, 6]]),
  array([[3, 1, 2],
         [4, 8, 5],
         [7, 0, 6]]),
  array([[3, 1, 2],
         [4, 8, 5],
         [7, 6, 0]]),
  array([[3, 1, 2],
         [4, 8, 0],
         [7, 6, 5]]),
  array([[3, 1, 2],
         [4, 0, 8],
         [7, 6, 5]]),
  array(

In [ ]:
unsolvable_state

array([[8, 1, 2],
       [0, 4, 3],
       [7, 6, 5]])

In [ ]:
greedy(unsolvable_state)

'The goal state is not reachable.'

# Trials

We create a random initial state, that is, a random permutation of the numbers `0, 1, 2, ..., 8` arranged in a 3 by 3 NumPy array.

In [ ]:
# This can be run to create a different initial state each time.
initial_state = np.random.permutation(numbers).reshape(3, 3)
initial_state

You can run both search algorithms and compare their performance. When we performed 20 trials, all of them (where the initial state was solvable) had similar results.

That is, BFS gave shorter solutions (paths to the goal state), but inspected a lot more nodes (in the hundred-thousands), while the greedy algorithm gave solutions of longer or equal length, but inspected way less nodes (in the hundreds).

This is because, in the search tree, BFS searches horizontally by "level" or distance from the initial state. So, when it stumbles upon a goal state, the path it took is guaranteed to be of minimum distance from the initial state. But, as it is an uninformed search strategy, it does not find the goal state efficiently, and so it ends up inspecting and expanding many nodes.

The greedy algorithm, on the other hand, searches by total manhattan distance, so it is not necessarily concerned if it "strays too far" from the initial state. But, since the total Manhattan distance is an appropriate representation of "how far" a state is from the goal state, the search is somewhat optimized and not as many nodes need to be inspected before the algorithm finds the goal state.

So, if one wishes to find the shortest possible path from an initial state to a goal state, BFS is the way to go. But, if one wants shorter computational time, the greedy algorithm will find a longer path but by inspecting a lot less nodes.

In [ ]:
bfs(initial_state)

In [ ]:
greedy(initial_state)